In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving array.npy to array.npy
User uploaded file "array.npy" with length 111788428 bytes


In [14]:
#@title Default title text
import os
PATH = os.getcwd()
data_dir_list = os.listdir(PATH) # show files which can be used in notebook
data_dir_list

['datalab',
 '.local',
 '.ipython',
 'test2.txt',
 'test.txt',
 'test3.txt',
 'array2.txt',
 '.forever',
 '.config',
 '.nv',
 '.rnd',
 '.cache']

In [0]:
from google.colab import files

files.download('array2.txt')

In [2]:
!pip install -U -q PyDrive
!pip install tqdm

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# PyDrive reference:
# https://googledrive.github.io/PyDrive/docs/build/html/index.html
#http://pythonhosted.org/PyDrive/pydrive.html



In [0]:
def fiload(q):
  """loads file into notebook
  q-string, name of a file in googledrive"""
  for file1 in file_list:
      if  file1 ["title"] == q:
        title=file1 ["title"]
        id=file1['id']
        file_obj = drive.CreateFile({'id': id})
        file_obj.GetContentFile(title)
        print("File is loaded!")
        break
      continue 

In [5]:
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList() #get list of all files in the root directory
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: Word2vec.ipynb, id: 1nKn7937mMcHB3nzj0st0kmJqfRKbe7bu
title: test3.txt, id: 1RQS6YAU32H1ULVkHmKdwOoO4ixxBRPtE
title: test2.txt, id: 1OaD91jnJHRd4QhqjDTz7rNAAEe5EkYHq
title: test.txt, id: 1w-fgsiP0vRm5jM67_CiudZQvNatP1SCy
title: Try to detect litho-new labels _RGB.ipynb, id: 1MSK9GHUDrpc9sQHzGW6Yw1FFG2lpYuF6
title: homework03_part2_vae.ipynb, id: 1KyRJ0UWQcYy7WSZOVPRTfBNNTrmtje76
title: humanparsing.py, id: 1XERUrbi0FeP0LvJR5XjlI4z5GD-eOSEG
title: lfw_dataset.py, id: 1347JBpZAfSd_0cn_U1UpYMEAHzzEIpXH
title: cifar.py, id: 14kQ1ll-sBcTR8LirL6eNIJDc8FlB0oyE
title: notmnist.py, id: 1aBL9iAo1a9a4thjZ_DQiusUiwFPFO4QS
title: homework_part1.ipynb, id: 1em1B6xFoJ3QCMvVAJxZbA1Sl2htUiWHn
title: Deep learning, id: 1lGgadZk68gbEx5qjRVZ9r41LdrKYtLNw
title: homework_part1.ipynb, id: 1LHZxE_Qnmz0zeBPOEvZsoQoh2w_SLpAJ
title: homework_differentiation.ipynb, id: 1dp1oYoR6GHbF8HJyOsos1ZSmfTXYFZN3
title: data.rar, id: 1KKqp6ZvEp6jsgeHvmTjzFcSctLyb25Ch
title: train_data_short_bw-512.npy, id: 1aqO7tr8n

In [6]:
name="test3.txt"
fiload(name)

File is loaded!


In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [8]:
!pip install tqdm

In [0]:
#model
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class SkipGramModel(nn.Module):
    """Skip gram model of word2vec.

    Attributes:
        emb_size: Embedding size.
        emb_dimention: Embedding dimention, typically from 50 to 500.
        u_embedding: Embedding for center word.
        v_embedding: Embedding for neibor words.
    """

    def __init__(self, emb_size, emb_dimension):
        """Initialize model parameters.

        Apply for two embedding layers.
        Initialize layer weight

        Args:
            emb_size: Embedding size.
            emb_dimention: Embedding dimention, typically from 50 to 500.

        Returns:
            None
        """
        super(SkipGramModel, self).__init__()
        self.emb_size = emb_size
        self.emb_dimension = emb_dimension
        self.u_embeddings = nn.Embedding(emb_size, emb_dimension, sparse=True)
        self.v_embeddings = nn.Embedding(emb_size, emb_dimension, sparse=True)
        self.init_emb()

    def init_emb(self):
        """Initialize embedding weight like word2vec.

        The u_embedding is a uniform distribution in [-0.5/em_size, 0.5/emb_size], and the elements of v_embedding are zeroes.

        Returns:
            None
        """
        initrange = 0.5 / self.emb_dimension
        self.u_embeddings.weight.data.uniform_(-initrange, initrange)
        self.v_embeddings.weight.data.uniform_(-0, 0)

    def forward(self, pos_u, pos_v, neg_v):
        """Forward process.

        As pytorch designed, all variables must be batch format, so all input of this method is a list of word id.

        Args:
            pos_u: list of center word ids for positive word pairs.
            pos_v: list of neibor word ids for positive word pairs.
            neg_u: list of center word ids for negative word pairs.
            neg_v: list of neibor word ids for negative word pairs.

        Returns:
            Loss of this process, a pytorch variable.
        """
        emb_u = self.u_embeddings(pos_u)
        emb_v = self.v_embeddings(pos_v)
        score = torch.mul(emb_u, emb_v).squeeze()
        score = torch.sum(score, dim=1)
        score = F.logsigmoid(score)
        neg_emb_v = self.v_embeddings(neg_v)
        neg_score = torch.bmm(neg_emb_v, emb_u.unsqueeze(2)).squeeze()
        neg_score = F.logsigmoid(-1 * neg_score)
        return -1 * (torch.sum(score)+torch.sum(neg_score))

    def save_embedding(self, id2word, file_name, use_cuda):
        """Save all embeddings to file.

        As this class only record word id, so the map from id to word has to be transfered from outside.

        Args:
            id2word: map from word id to word.
            file_name: file name.
        Returns:
            None.
        """
        if use_cuda:
            embedding = self.u_embeddings.weight.cpu().data.numpy()
        else:
            embedding = self.u_embeddings.weight.data.numpy()
        fout = open(file_name, 'w')
        fout.write('%d %d\n' % (len(id2word), self.emb_dimension))
        for wid, w in id2word.items():
            e = embedding[wid]
            e = ' '.join(map(lambda x: str(x), e))
            fout.write('%s %s\n' % (w, e))


def test():
    model = SkipGramModel(100, 100)
    id2word = dict()
    for i in range(100):
        id2word[i] = str(i)
    model.save_embedding(id2word)


#if __name__ == '__main__':
 #   test()


In [0]:
#input data 
import numpy
from collections import deque
numpy.random.seed(12345)


class InputData:
    """Store data for word2vec, such as word map, sampling table and so on.

    Attributes:
        word_frequency: Count of each word, used for filtering low-frequency words and sampling table
        word2id: Map from word to word id, without low-frequency words.
        id2word: Map from word id to word, without low-frequency words.
        sentence_count: Sentence count in files.
        word_count: Word count in files, without low-frequency words.
    """

    def __init__(self, file_name, min_count):
        self.input_file_name = file_name
        self.get_words(min_count)
        self.word_pair_catch = deque()
        self.init_sample_table()
        print('Word Count: %d' % len(self.word2id))
        print('Sentence Length: %d' % (self.sentence_length))

    def get_words(self, min_count):
        self.input_file = open(self.input_file_name)
        self.sentence_length = 0
        self.sentence_count = 0
        word_frequency = dict()
        for line in self.input_file:
            self.sentence_count += 1
            line = line.strip().split(' ')
            self.sentence_length += len(line)
            for w in line:
                try:
                    word_frequency[w] += 1
                except:
                    word_frequency[w] = 1
        self.word2id = dict()
        self.id2word = dict()
        wid = 0
        self.word_frequency = dict()
        for w, c in word_frequency.items():
            if c < min_count:
                self.sentence_length -= c
                continue
            self.word2id[w] = wid
            self.id2word[wid] = w
            self.word_frequency[wid] = c
            wid += 1
        self.word_count = len(self.word2id)

    def init_sample_table(self):
        self.sample_table = []
        sample_table_size = 1e8
        pow_frequency = numpy.array(list(self.word_frequency.values()))**0.75
        words_pow = sum(pow_frequency)
        ratio = pow_frequency / words_pow
        count = numpy.round(ratio * sample_table_size)
        for wid, c in enumerate(count):
            self.sample_table += [wid] * int(c)
        self.sample_table = numpy.array(self.sample_table)

    # @profile
    def get_batch_pairs(self, batch_size, window_size):
        while len(self.word_pair_catch) < batch_size:
            sentence = self.input_file.readline()
            if sentence is None or sentence == '':
                self.input_file = open(self.input_file_name)
                sentence = self.input_file.readline()
            word_ids = []
            for word in sentence.strip().split(' '):
                try:
                    word_ids.append(self.word2id[word])
                except:
                    continue
            for i, u in enumerate(word_ids):
                for j, v in enumerate(
                        word_ids[max(i - window_size, 0):i + window_size]):
                    assert u < self.word_count
                    assert v < self.word_count
                    if i == j:
                        continue
                    self.word_pair_catch.append((u, v))
        batch_pairs = []
        for _ in range(batch_size):
            batch_pairs.append(self.word_pair_catch.popleft())
        return batch_pairs

    # @profile
    def get_neg_v_neg_sampling(self, pos_word_pair, count):
        neg_v = numpy.random.choice(
            self.sample_table, size=(len(pos_word_pair), count)).tolist()
        return neg_v

    def evaluate_pair_count(self, window_size):
        return self.sentence_length * (2 * window_size - 1) - (
            self.sentence_count - 1) * (1 + window_size) * window_size


def test():
    a = InputData('./zhihu.txt')


#if __name__ == '__main__':
 #   test()


In [11]:
#w2vec
import numpy
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import sys


class Word2Vec:
    def __init__(self,
                 input_file_name="zhihu2.txt",
                 output_file_name="zhihuvec.txt",
                 emb_dimension=100,
                 batch_size=50,
                 window_size=5,
                 iteration=1,
                 initial_lr=0.025,
                 min_count=5):
        """Initilize class parameters.

        Args:
            input_file_name: Name of a text data from file. Each line is a sentence splited with space.
            output_file_name: Name of the final embedding file.
            emb_dimention: Embedding dimention, typically from 50 to 500.
            batch_size: The count of word pairs for one forward.
            window_size: Max skip length between words.
            iteration: Control the multiple training iterations.
            initial_lr: Initial learning rate.
            min_count: The minimal word frequency, words with lower frequency will be filtered.

        Returns:
            None.
        """
        self.data = InputData(input_file_name, min_count)
        self.output_file_name = output_file_name
        self.emb_size = len(self.data.word2id)
        self.emb_dimension = emb_dimension
        self.batch_size = batch_size
        self.window_size = window_size
        self.iteration = iteration
        self.initial_lr = initial_lr
        self.skip_gram_model = SkipGramModel(self.emb_size, self.emb_dimension)
        self.use_cuda = torch.cuda.is_available()
        if self.use_cuda:
            self.skip_gram_model.cuda()
        self.optimizer = optim.SGD(
            self.skip_gram_model.parameters(), lr=self.initial_lr)

    def train(self):
        """Multiple training.

        Returns:
            None.
        """
        pair_count = self.data.evaluate_pair_count(self.window_size)
        batch_count = self.iteration * pair_count / self.batch_size
        process_bar = tqdm(range(int(batch_count)))
        # self.skip_gram_model.save_embedding(
        #     self.data.id2word, 'begin_embedding.txt', self.use_cuda)
        for i in process_bar:
            pos_pairs = self.data.get_batch_pairs(self.batch_size,
                                                  self.window_size)
            neg_v = self.data.get_neg_v_neg_sampling(pos_pairs, 5)
            pos_u = [pair[0] for pair in pos_pairs]
            pos_v = [pair[1] for pair in pos_pairs]

            pos_u = Variable(torch.LongTensor(pos_u))
            pos_v = Variable(torch.LongTensor(pos_v))
            neg_v = Variable(torch.LongTensor(neg_v))
            if self.use_cuda:
                pos_u = pos_u.cuda()
                pos_v = pos_v.cuda()
                neg_v = neg_v.cuda()

            self.optimizer.zero_grad()
            loss = self.skip_gram_model.forward(pos_u, pos_v, neg_v)
            loss.backward()
            self.optimizer.step()

            process_bar.set_description("Loss: %0.8f, lr: %0.6f" %
                                        (loss.data[0],
                                         self.optimizer.param_groups[0]['lr']))
            if i * self.batch_size % 100000 == 0:
                lr = self.initial_lr * (1.0 - 1.0 * i / batch_count)
                for param_group in self.optimizer.param_groups:
                    param_group['lr'] = lr
        self.skip_gram_model.save_embedding(
            self.data.id2word, self.output_file_name, self.use_cuda)


if __name__ == '__main__':
    w2v = Word2Vec(input_file_name="test3.txt", output_file_name="array2.txt")
    w2v.train()


Word Count: 321
Sentence Length: 4172


Loss: 135.35122681, lr: 0.025000: 100%|██████████| 750/750 [00:06<00:00, 110.79it/s]


In [5]:
!pip install tqdm

    100% |████████████████████████████████| 51kB 2.0MB/s 


In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [15]:
#test - didn't use this one
import sys
from sklearn.metrics.pairwise import cosine_similarity
import numpy
f=open("test3.txt")
f.readline()
all_embeddings=[]
all_words=[]
word2id=dict()
for i,line in enumerate(f):
    line=line.strip().split(' ')
    word=line[0]
    embedding=[float(x) for x in line[1:]]
    assert len(embedding)==100
    all_embeddings.append(embedding)
    all_words.append(word)
    word2id[word]=i
all_embeddings=numpy.array(all_embeddings)
while 1:
    word=input('Word: ')
    try:
        wid=word2id[word]
    except:
        print('Cannot find this word')
        continue
    embedding=all_embeddings[wid:wid+1]
    d = cosine_similarity(embedding, all_embeddings)[0]
    d=zip(all_words, d)
    d=sorted(d, key=lambda x:x[1], reverse=True)
    for w in d[:10]:
        if len(w[0])<2:
            continue
        print(w)


Word: TAAC 
Cannot find this word


KeyboardInterrupt: ignored

In [18]:
i

0

In [0]:
all_embeddings